In [1]:
import pandas as pd

In [3]:
# Шаг 1: Загрузите полный CSV-файл в DataFrame
file_path = "/mnt/d/Doc_To_Date/20240506/additional_results.csv"
df = pd.read_csv(file_path)

In [6]:
# Шаг 2: Определите количество строк и размер каждой части
total_rows = len(df)
num_parts = 3
chunk_size = total_rows // num_parts

In [7]:
# Шаг 3: Разделите DataFrame на три части
chunks = [df[i*chunk_size:(i+1)*chunk_size] for i in range(num_parts)]

In [8]:
# Последний кусок может содержать остаток, если общее количество строк не кратно числу частей
if total_rows % num_parts != 0:
    chunks[-1] = pd.concat([chunks[-1], df[num_parts * chunk_size:]])

In [9]:
# Шаг 4: Сохраните каждую часть как отдельный CSV-файл
for i, chunk in enumerate(chunks):
    chunk.to_csv(f"additional_results_part_{i+1}.csv", index=False)

Для АСНА переименование файлов и изменение дилиметра


In [13]:
import pandas as pd

# Определение пути к исходным файлам в WSL
path = "/mnt/d/АСНА/20240710/new_load_2/archive-2024-07-10_20-03-16 (АСНА АПРЕЛЬ МАЙ ИЮНЬ)_new/"
files = [
    "asna_dump_2024_07_10_1_new.csv", 
    "asna_dump_2024_07_10_2_new.csv",
    "asna_dump_2024_07_10_3_new.csv",
    "asna_dump_2024_07_10_4_new.csv"
]

In [14]:
# Новые имена для сохранения файлов
new_names = [
    "asna-nomenclature-coefs-1.csv",
    "asna-nomenclature-coefs-2.csv",
    "asna-nomenclature-coefs-3.csv",
    "asna-nomenclature-coefs-4.csv"
]

In [15]:

# Загрузка, обработка (если необходимо) и сохранение файлов
for old_name, new_name in zip(files, new_names):
    try:
        # Загрузка с правильным разделителем и указанием кодировки
        df = pd.read_csv(path + old_name, delimiter=';', encoding='utf-8')
    except UnicodeDecodeError:
        # Если возникает ошибка декодирования, попробуйте другую кодировку
        df = pd.read_csv(path + old_name, delimiter=';', encoding='latin1')
    
    # Сохранение с изменением разделителя на запятую, если это необходимо
    df.to_csv(path + new_name, index=False, sep=',')

_________________


In [3]:
import os
import pandas as pd

# Укажи путь к папке с файлами
folder_path = '/mnt/d/Doc_To_Date/20240904'  # Замените на ваш путь к папке

# Получаем список всех файлов CSV в папке
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Создаем пустой DataFrame для объединения всех файлов
combined_df = pd.DataFrame()

# Читаем и объединяем все файлы
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path, sep = '|')
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Сохраняем объединенные данные в новый CSV файл (по желанию)
combined_df.to_csv('/mnt/d/Doc_To_Date/20240904/combined_file.csv', index=False)

# Если нужен поиск по ID, укажи здесь, что нужно делать дальше
# Например, поиск по определенной колонке:
# id_column = 'id'  # Название колонки с ID
# unique_ids = combined_df[id_column].unique()
# print(f"Уникальные ID: {unique_ids}")


In [1]:
import psycopg2

def connect_to_database():
    conn = psycopg2.connect(
        dbname='pharmhub',
        user='ivan.anisimov',
        password='oA7W5uz2GzEd',
        host='10.0.0.170'
    )
    return conn

# Проверим подключение
conn = connect_to_database()
print("Успешно подключено к базе данных") if conn else print("Ошибка подключения")

Успешно подключено к базе данных


In [2]:
import pandas as pd

# Чтение CSV файла
file_path = '/mnt/d/Doc_To_Date/20240904/combined_file.csv'  # Укажи здесь правильный путь к файлу
df = pd.read_csv(file_path)

# Получаем список значений из колонки 'title'
titles = df['title'].tolist()
titles[:5]  # Покажем первые 5 значений для проверки


['АГАРТА 50 МГ №30 ТАБ',
 'АГАРТА МЕТ 50 МГ + 850 МГ №60 ТАБ',
 'АГАРТА МЕТ 50 МГ + 1000 МГ №60 ТАБ',
 'СИТАДИАБ 100 МГ №28 ТАБ',
 'ЛИНДИНЕТ ТАБ 75 МКГ 30 МКГ №21']

In [3]:
# Формируем SQL-запрос с использованием списка значений
def search_in_database(conn, titles):
    # Формируем часть запроса для LIKE ANY (ARRAY [...])
    like_conditions = ["%{}%".format(title) for title in titles]

    query = """
    SELECT MSU.id,
           MSU."externalId",
           MSU.title,
           C.title AS client_title,
           C.id AS client_id,
           MSU."EAN",
           MSU.status,
           MSU."updatedAt",
           MSU.data,
           MSU."updateIncrement"
    FROM "ManufSKUs" MSU
    JOIN "Clients" C ON MSU.client = C.id
    WHERE MSU.title ILIKE ANY (ARRAY[{}])
    ORDER BY MSU.title;
    """.format(", ".join(["%s"] * len(like_conditions)))

    with conn.cursor() as cur:
        cur.execute(query, like_conditions)
        results = cur.fetchall()
        return results

# Выполняем запрос к базе данных
results = search_in_database(conn, titles)

# Выводим результаты
for row in results:
    print(f"ID: {row[0]}, Title: {row[2]}")

ID: 31068, Title: 00351167160404
ID: 31067, Title: 00351167160503
ID: 31034, Title: 03582910071662
ID: 31024, Title: 03582910089339
ID: 31056, Title: 03582910090663
ID: 31053, Title: 03582910090670
ID: 31064, Title: 03582910091035
ID: 31026, Title: 03664798002317
ID: 31057, Title: 03664798002324
ID: 31055, Title: 03664798011753
ID: 31048, Title: 03664798021882
ID: 31029, Title: 03664798036305
ID: 31051, Title: 03664798038798
ID: 32246, Title: 03664798040814
ID: 32242, Title: 03664798040821
ID: 32243, Title: 03664798040838
ID: 32244, Title: 03664798042306
ID: 32188, Title: 03664798043877
ID: 31035, Title: 03664798045154
ID: 31022, Title: 03664798045307
ID: 31054, Title: 03664798045314
ID: 31033, Title: 03664798047028
ID: 31021, Title: 03664798047127
ID: 31044, Title: 03664798047448
ID: 31066, Title: 03664798047455
ID: 31065, Title: 03664798047974
ID: 31032, Title: 03664798048643
ID: 31023, Title: 03664798050219
ID: 31060, Title: 03664798050226
ID: 31930, Title: 03664798051766
ID: 31017,

In [6]:
import pandas as pd

# Преобразуем результаты запроса в DataFrame
def results_to_dataframe(results):
    # Предположим, что запрос возвращает колонки в следующем порядке:
    columns = ['ID', 'External ID', 'Title', 'Client Title', 'Client ID', 'EAN', 'Status', 'Updated At', 'Data', 'Update Increment']
    
    # Создаем DataFrame из результатов
    df_results = pd.DataFrame(results, columns=columns)
    
    return df_results

# Преобразуем результаты в DataFrame
df_results = results_to_dataframe(results)

# Выводим первые 5 строк для проверки
df_results


,ID,External ID,Title,Client Title,Client ID,EAN,Status,Updated At,Data,Update Increment
0,31068,00351167160404,00351167160404,Опелла,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,0351167160404,clarification,2024-04-10 16:16:25.348552,"{'shortTitle': '', 'parsedTitle': {'articul': ...",59009
1,31067,00351167160503,00351167160503,Опелла,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,0351167160503,clarification,2024-04-08 16:17:45.024335,"{'shortTitle': '', 'parsedTitle': {'articul': ...",59008
2,31034,03582910071662,03582910071662,Опелла,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,None,foreign,2024-06-25 11:32:54.331916,{'manuf': '5a2d6114-b82a-4ac2-83ae-12ccc304f6f...,61523
3,31024,03582910089339,03582910089339,Опелла,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,None,foreign,2024-06-25 11:32:54.331916,{'manuf': '5a2d6114-b82a-4ac2-83ae-12ccc304f6f...,61501
4,31056,03582910090663,03582910090663,Опелла,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,None,foreign,2024-06-25 11:32:54.331916,{'manuf': '5a2d6114-b82a-4ac2-83ae-12ccc304f6f...,61569
...,...,...,...,...,...,...,...,...,...,...
2905,30435,яя д-сан для взрослых фл. 10 мл (бад),яя Д-САН для взрослых фл. 10 мл (БАД),Опелла,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,,foreign,2024-07-12 13:00:19.156128,{'manuf': '5a2d6114-b82a-4ac2-83ae-12ccc304f6f...,64993
2906,29773,яя д-сан таб. жеват. массой 120 мг №100,яя Д-САН таб. жеват. массой 120 мг №100,Опелла,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,,foreign,2024-07-12 13:00:19.156128,{'manuf': '5a2d6114-b82a-4ac2-83ae-12ccc304f6f...,64181
2907,30219,"яя**но-шпа таб. 0,04 №60 флак/доз","яя**Но-шпа таб. 0,04 №60 флак/доз",Опелла,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,,foreign,2024-07-12 13:00:19.156128,{'manuf': '5a2d6114-b82a-4ac2-83ae-12ccc304f6f...,64713
2908,30495,"яя**эссенциале форте-н капс. 0,3 №100","яя**Эссенциале форте-Н капс. 0,3 №100",Опелла,5a2d6114-b82a-4ac2-83ae-12ccc304f6fb,,foreign,2024-07-12 13:00:19.156128,{'manuf': '5a2d6114-b82a-4ac2-83ae-12ccc304f6f...,65075


In [7]:
import psycopg2

def connect_to_database():
    conn = psycopg2.connect(
        dbname='pharmhub',
        user='ivan.anisimov',
        password='oA7W5uz2GzEd',
        host='10.0.1.4'
    )
    return conn

# Проверим подключение
conn = connect_to_database()
print("Успешно подключено к базе данных") if conn else print("Ошибка подключения")

OperationalError: connection to server at "10.0.1.4", port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


In [11]:
import vertica_python

# Функция для подключения к базе данных Vertica
def connect_to_vertica():
    conn_info = {
        'host': '10.0.1.4',
        'port': 5433,  
        'user': 'ivan.anisimov',
        'password': 'oA7W5uz2GzEd',  
        'autocommit': True
    }

    # Подключаемся к базе данных
    connection = vertica_python.connect(**conn_info)
    return connection

# Проверим подключение
try:
    conn = connect_to_vertica()
    print("Успешно подключено к базе данных Vertica")
except Exception as e:
    print(f"Ошибка подключения: {e}")

Успешно подключено к базе данных Vertica


In [15]:
# Извлекаем все SKU ID из DataFrame (предположим, колонка называется 'ID')
sku_id_list = df_results['ID'].tolist()  # Если колонка с ID называется иначе, поправь здесь

# Преобразуем список SKU ID в строку, которую можно вставить в SQL запрос
sku_id_str = ','.join([str(sku_id) for sku_id in sku_id_list])

# SQL запрос с динамически вставленными SKU ID
query = f"""
    SELECT skuId, SUM(sum)::int, SUM(quantity)::int
    FROM Checks
    WHERE docDate >= '2024-01-01'
    AND ofdCredId > 0
    AND skuId IN ({sku_id_str})
    GROUP BY skuId;
"""

# Выполняем запрос
def execute_query_with_sku_ids(conn, query):
    with conn.cursor() as cur:
        cur.execute(query)
        results = cur.fetchall()
        return results

# Пример использования
try:
    conn = connect_to_vertica()  # Предполагается, что функция подключения к Vertica уже написана
    results = execute_query_with_sku_ids(conn, query)

    # Выводим результаты
    for row in results:
        print(f"SKU ID: {row[0]}, Sum: {row[1]}, Quantity: {row[2]}")
    
    # Преобразуем результаты в DataFrame, если нужно
    df_query_results = pd.DataFrame(results, columns=['skuId', 'Sum', 'Quantity'])
    
    # Сохраняем результаты в CSV, если нужно
    df_query_results.to_csv('/mnt/d/Doc_To_Date/20240904/combined_file_2.csv', index=False)
    print("Результаты сохранены в файл checks_results.csv")
    
finally:
    conn.close()  # Закрываем соединение с базой данных

SKU ID: 26193, Sum: 8471, Quantity: 28
SKU ID: 28143, Sum: 311399, Quantity: 1520
SKU ID: 29463, Sum: 11604273, Quantity: 12552
SKU ID: 29493, Sum: 745, Quantity: 3
SKU ID: 29503, Sum: 67832, Quantity: 227
SKU ID: 29553, Sum: 17835, Quantity: 30
SKU ID: 29563, Sum: 69716, Quantity: 1383
SKU ID: 29583, Sum: 37212, Quantity: 179
SKU ID: 30623, Sum: 2238, Quantity: 8
SKU ID: 30703, Sum: 263896, Quantity: 910
SKU ID: 593, Sum: 2136, Quantity: 4
SKU ID: 21473, Sum: 241544, Quantity: 588
SKU ID: 32003, Sum: 12002, Quantity: 54
SKU ID: 32123, Sum: 375331, Quantity: 1243
SKU ID: 32133, Sum: 160344, Quantity: 392
SKU ID: 32153, Sum: 220436, Quantity: 648
SKU ID: 23743, Sum: 507180, Quantity: 1294
SKU ID: 35043, Sum: 1866, Quantity: 8
SKU ID: 35053, Sum: 86499, Quantity: 248
SKU ID: 4593, Sum: 1064, Quantity: 7
SKU ID: 26790, Sum: 23033, Quantity: 70
SKU ID: 29460, Sum: 537347, Quantity: 286
SKU ID: 29470, Sum: 13129, Quantity: 72
SKU ID: 29480, Sum: 2181, Quantity: 12
SKU ID: 29490, Sum: 344565